In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# load data
train_path, test_path = "../input/titanic/train.csv", "../input/titanic/test.csv"
train_df = pd.read_csv(train_path)
test_df  = pd.read_csv(test_path)
train_df.info()

# merge train test dataset and assign y
main_df = pd.concat([train_df,test_df])
y = train_df.Survived

In [ ]:
print(main_df.groupby("Pclass".split()).agg(["count","sum","mean"])["Age"])
temp_mean = main_df.groupby("Pclass".split()).agg(["count","sum","mean"])["Age"]["mean"].tolist()

def fill_age(cols, mean_list):
    pclass, age = cols[0], cols[1]
    if pd.isnull(age):
        for i in range(1,4):
            if pclass == i:
                age = mean_list[i-1]
    else:
        age = cols[1]
    return age

main_df.Age = main_df["Pclass Age".split()].apply(fill_age, axis=1, args=(temp_mean,))
train_df.Age = train_df["Pclass Age".split()].apply(fill_age, axis=1, args=(temp_mean,))
test_df.Age = test_df["Pclass Age".split()].apply(fill_age, axis=1, args=(temp_mean,))

def trans_age(age):
    temp=None
    if age <= 2:
        temp = 1
    elif 2 < age <= 12:
        temp = 2
    elif 12 < age <= 18:
        temp = 3
    elif 18 < age <= 40:
        temp = 4
    elif 40 < age <= 60:
        temp = 5
    else:
        temp = 6
    return temp

main_df["agegroup"] = main_df.Age.apply(trans_age)

In [ ]:
main_df["last_name"] = main_df.Name.apply(lambda x: x.split(",")[0])
main_df["title_name"] = main_df.Name.apply(lambda x: x.split(",")[1].split(".")[0])

In [ ]:
# fillna Fare by get mean of groupby(pclass, agegroup)
# print(main_df.groupby("Pclass agegroup".split()).agg("count sum mean".split())["Fare"])
# print(main_df[main_df.Fare.isnull()])
temp = main_df.groupby("Pclass agegroup".split()).agg("mean",)["Fare"]
temp = {temp.index[i]: t for i, t in enumerate(temp)}
def fill_fare(cols, temp):
    if pd.isnull(cols[2]):
        for k,v in temp.items():
            if (cols[0], cols[1]) == k:
                return temp[k]
    else:
        return cols[2]
main_df.Fare = main_df["Pclass agegroup Fare".split()].apply(fill_fare, axis=1, args=(temp,))

In [ ]:
main_df.last_name.value_counts()

In [ ]:
main_df["family_size"] = main_df.SibSp + main_df.Parch + 1

In [ ]:
# cabin_list = sorted(list(set(" ".join(main_df.Cabin[main_df.Cabin.isnull()==False].tolist()).split())))
# cabin_list
# for col in cabin_list:
#     main_df[col] = 0
# indexes = main_df[main_df.Cabin.isnull()==False].index
# for i in indexes:
#     value = main_df.iloc[i].Cabin
#     if pd.isnull(value):
#         pass
#     elif type(value)==str:
#         value=value.split()
#         if len(value)>1:
#             for s in value:
#                 main_df.loc[i,s] = 1
#         else:
#             main_df.loc[i,value] =1

In [ ]:
main_df = pd.concat([main_df.drop(["Sex"],axis=1),pd.get_dummies(main_df.Sex, drop_first=True)],axis=1)
main_df = pd.concat([main_df.drop(["Embarked"],axis=1),pd.get_dummies(main_df.Embarked)],axis=1)
main_df = main_df.drop(["Name","Cabin"],axis=1)

In [ ]:
df = pd.DataFrame({"cnull":main_df.isnull().sum()})
df[df.cnull>0]

In [ ]:
for i in "title_name last_name Ticket".split():
    main_df[i] = main_df[i].astype("category").cat.codes

In [ ]:
main_df.columns[:20]

In [ ]:
df = pd.DataFrame({"cnull":main_df.isnull().sum()})
df[df.cnull>0]

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix

X = main_df[:len(train_df)].drop(["Survived"],axis=1)
y = main_df[:len(train_df)].Survived
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
model = RandomForestRegressor(n_estimators=100, random_state=0)
model.fit(X_train,y_train)
print(classification_report(model.predict(X_test).round(),y_test))
print(pd.DataFrame({"ori":model.predict(X_test), "round":model.predict(X_test).round(), "y":y_test}))
pred = confusion_matrix(model.predict(X_test).round().astype("int"),y_test)
print("Accuracy",sum(pred)[0]/(sum(pred)[0]+sum(pred)[1]))

In [ ]:
main_df = main_df.drop(["Survived"],axis=1)
model = RandomForestRegressor(n_estimators=100, random_state=1)
model.fit(X,y)
submit = pd.DataFrame({"PassengerId":main_df[len(train_df):].PassengerId, "Survived":model.predict(main_df[len(train_df):]).round()})
submit.Survived = submit.Survived.astype("int")
submit.to_csv("submissionJavier_Vallejos.csv",index = False)

In [ ]:
pd.concat([main_df[:len(train_df)].Fare, train_df.Fare],axis=1)